# Projeto de Bases de Dados - Parte 3

### Docente Responsável

Prof. FirstName LastName

### Grupo GG
<dl>
    <dt>HH horas (33.3%)</dt>
    <dd>istxxxxxxxx FirstName LastName</dd>
    <dt>HH horas (33.3%)</dt>
    <dd>istxxxxxxxx FirstName LastName</dd>
    <dt>HH horas (33.3%)</dt>
    <dd>istxxxxxxxx FirstName LastName</dd>
<dl>

In [ ]:
%load_ext sql
%sql postgresql://db:db@postgres/db

# Empresa de comércio online

## 0. Carregamento da Base de Dados

Carregue o esquema de Base de Dados apresentado no Anexo A.

In [ ]:
%%sql

DROP TABLE IF EXISTS customer CASCADE;
DROP TABLE IF EXISTS orders CASCADE;
DROP TABLE IF EXISTS pay CASCADE;
DROP TABLE IF EXISTS employee CASCADE;
DROP TABLE IF EXISTS process CASCADE;
DROP TABLE IF EXISTS department CASCADE;
DROP TABLE IF EXISTS workplace CASCADE;
DROP TABLE IF EXISTS works CASCADE;
DROP TABLE IF EXISTS office CASCADE;
DROP TABLE IF EXISTS warehouse CASCADE;
DROP TABLE IF EXISTS product CASCADE;
DROP TABLE IF EXISTS contains CASCADE;
DROP TABLE IF EXISTS supplier CASCADE;
DROP TABLE IF EXISTS delivery CASCADE;

CREATE TABLE customer(
cust_no INTEGER PRIMARY KEY,
name VARCHAR(80) NOT NULL,
email VARCHAR(254) UNIQUE NOT NULL,
phone VARCHAR(15),
address VARCHAR(255)
);

CREATE TABLE orders(
order_no INTEGER PRIMARY KEY,
cust_no INTEGER NOT NULL REFERENCES customer,
date DATE NOT NULL
--order_no must exist in contains
);

CREATE TABLE pay(
order_no INTEGER PRIMARY KEY REFERENCES orders,
cust_no INTEGER NOT NULL REFERENCES customer
);

CREATE TABLE employee(
ssn VARCHAR(20) PRIMARY KEY,
TIN VARCHAR(20) UNIQUE NOT NULL,
bdate DATE,
name VARCHAR NOT NULL
--age must be >=18
);

CREATE TABLE process(
ssn VARCHAR(20) REFERENCES employee,
order_no INTEGER REFERENCES orders,
PRIMARY KEY (ssn, order_no)
);

CREATE TABLE department(
name VARCHAR PRIMARY KEY
);

CREATE TABLE workplace(
address VARCHAR PRIMARY KEY,
lat NUMERIC(8, 6) NOT NULL,
long NUMERIC(9, 6) NOT NULL,
UNIQUE(lat, long)
--address must be in warehouse or office but not both
);

CREATE TABLE office(
address VARCHAR(255) PRIMARY KEY REFERENCES workplace
);

CREATE TABLE warehouse(
address VARCHAR(255) PRIMARY KEY REFERENCES workplace
);

CREATE TABLE works(
ssn VARCHAR(20) REFERENCES employee,
name VARCHAR(200) REFERENCES department,
address VARCHAR(255) REFERENCES workplace,
PRIMARY KEY (ssn, name, address)
);

CREATE TABLE product(
SKU VARCHAR(25) PRIMARY KEY,
name VARCHAR(200) NOT NULL,
description VARCHAR,
price NUMERIC(10, 2) NOT NULL,
ean NUMERIC(13) UNIQUE
);

CREATE TABLE contains(
order_no INTEGER REFERENCES orders,
SKU VARCHAR(25) REFERENCES product,
qty INTEGER,
PRIMARY KEY (order_no, SKU)
);

CREATE TABLE supplier(
TIN VARCHAR(20) PRIMARY KEY,
name VARCHAR(200),
address VARCHAR(255),
SKU VARCHAR(25) REFERENCES product,
date DATE
);

CREATE TABLE delivery(
address VARCHAR(255) REFERENCES warehouse,
TIN VARCHAR(20) REFERENCES supplier,
PRIMARY KEY (address, TIN)
);

Crie as instruções para o seu preenchimento de forma consistente, garantindo que todas as consultas SQL e OLAP, apresentadas mais adiante, produzam um resultado não vazio. 

In [ ]:
%%sql

INSERT INTO customer (cust_no, name, email, phone, address)
VALUES
    (1, 'Manuel Mendonça', 'manuelmendoncag@gmail.com', 913454293, 'Rua das Forças Armadas, 3, 2805-313 Almada'),
    (2, 'José Figueiras', 'jose@sic.pt', '913923466', 'Rua das Maçãs, 4, 2750-177 Cascais'),
    (3, 'Catarina Destapado', 'catdestapado@gmail.com', 924374293, 'Avenida dos Aliados, 15, 1950-322 Lisboa'),
    (4, 'Diana Figueiras', 'diana.f@sapo.pt', '953928436', 'Rua das Maçãs, 4, 2750-177 Cascais'),
    (5, 'Asdrúbal Vieira', 'asdrubalvieira@hotmail.com', 978374827, 'Rua dos Bacalhoeiros, 7, 2805-450 Almada'),
    (6, 'Zé Figueiras', 'ze@gmail.pt', '913923466', 'Rua das Maçãs, 4, 2750-177 Cascais'),
    (7, 'Luísa Martins', 'lulu2001@gmail.com', 932842839, 'Rua Luis Simões Lisboa, 10, 2740-242 Oeiras'),
    (8, 'António Silva', 'a.silva@gmail.com', '924399979', 'Rua do 25 de Abril, 16, 2660-503 Loures');

INSERT INTO orders (order_no, cust_no, date)
VALUES
    (1, 3, '2021-12-23'),
    (2, 3, '2022-02-14'),
    (3, 5, '2022-02-14'),
    (4, 1, '2022-04-30'),
    (5, 2, '2022-07-02'),
    (6, 4, '2022-08-02'),
    (7, 2, '2022-10-21'),
    (8, 7, '2022-12-26'),
    (9, 1, '2022-12-26'),
    (10, 6, '2023-01-21'),
    (11, 8, '2023-05-03'),
    (12, 2, '2023-06-01');

INSERT INTO pay (order_no, cust_no)
VALUES
    (1, 3),
    (4, 1),
    (11, 7),
    (12, 1),
    (5, 2),
    (7, 2),
    (9, 8);

INSERT INTO employee (ssn, TIN, bdate, name)
VALUES
    ('47384739204', '264728040', '1974-04-20', 'Leonor Mateus'),
    ('12212342932', '230140203', '1980-01-17', 'Luís Raposo'),
    ('38447293028', '493850384', '1978-03-04', 'Patrícia Días'),
    ('83892403759', '382940293', '1993-08-23', 'Inês Santos'),
    ('22345678931', '837402837', '1982-01-01', 'Xico Tinense'),
    ('20343676821', '382938374', '1990-01-01', 'Sheik Malik'),
    ('19342678712', '283927492', '2000-01-01', 'Messinaldo da Silva'),
    ('19845678612', '182749192', '2003-01-01', 'Taria Mumaz');

INSERT INTO process (ssn, order_no)
VALUES
    ('47384739204', 1),
    ('83892403759', 2),
    ('47384739204', 2),
    ('12212342932', 4),
    ('12212342932', 3),
    ('22345678931', 5),
    ('12212342932', 5),
    ('12212342932', 6),
    ('20343676821', 6),
    ('19342678712', 7),
    ('12212342932', 7),
    ('12212342932', 8),
    ('19845678612', 8),
    ('22345678931', 9),
    ('20343676821', 10),
    ('12212342932', 11),
    ('38447293028', 12);

INSERT INTO department (name)
VALUES
    ('Marketing'),
    ('Sales'),
    ('HR'),
    ('IT');

INSERT INTO workplace (address, lat, long)
VALUES
    ('Rua dos Descobrimentos, 17, 2750-177 Cascais', 38.697089, -9.421686),
    ('Rua da Amizade, 5, 2660-503 Loures', 38.830518, -9.168244),
    ('Rua Dr. Eduardo Neves, 9, Lisboa', 38.698483, -9.421928);

INSERT INTO office (address)
VALUES
    ('Rua dos Descobrimentos, 17, 2750-177 Cascais'),
    ('Rua Dr. Eduardo Neves, 9, Lisboa');

INSERT INTO warehouse (address)
VALUES
    ('Rua da Amizade, 5, 2660-503 Loures');

INSERT INTO works (ssn, name, address)
VALUES
    ('47384739204', 'Marketing', 'Rua dos Descobrimentos, 17, 2750-177 Cascais'),
    ('12212342932', 'Sales', 'Rua Dr. Eduardo Neves, 9, Lisboa'),
    ('38447293028', 'IT', 'Rua dos Descobrimentos, 17, 2750-177 Cascais'),
    ('83892403759', 'Sales', 'Rua da Amizade, 5, 2660-503 Loures');

INSERT INTO product (SKU, name, description, price, ean)
VALUES
    ('SKU00001', 'VAN GOGH Aguarela', 'A aguarela Van Gogh é uma tinta transparente com cores intensas.',
    6.59, 5601234567890),
    ('SKU00002', 'WINSOR NEWTON Aguarela', 'As aguarelas da gama Cotman são produzidas dentro dos elevados padrões de qualidade da Winsor & Newton, na qual oferecem uma ótima relação de qualidade - preço, isto por alguns dos pigmentos mais caros serem substituídos por alternativas menos dispendiosas como pigmentos tradicionais e tintas sintéticas de boa qualidade.',
    4.49, 5909876543216),
    ('SKU00003', 'REXEL Destruidora', 'A gama de destruidores Secure da Rexel é ideal para destruir documentos pessoais e informações confidenciais. A destruidora Secure MC4SL destrói até 4 folhas de papel A4 (80 g/m²) de uma vez, com nível de segurança P-5 (2x15mm).',
    219.00, 4001122334455),
    ('SKU00004', 'MAKENOTES Porta Óculos Patio, Verde', 'Porta Óculos da MAKENOTES, coleção Patio, com fecho de mola na parte superior. Feito em pele sintética impressa eco-friendly (sem origem animal e livre de ftalatos) e é também repelente à água.',
    8.95, 8712345678902),
    ('SKU00005', 'TIMEOFFICE Separadores de Índice AZ/160/4, Cartolina, 20 Separadores, A4, Bege', 'Separadores de índice da TIMEOFFICE, com 20 separadores em cartolina de 160g/m², com 4 furos e formato A4.',
    2.99, 9345678901234),
    ('SKU00006', 'BI-OFFICE Quadro Branco, Superfície Magnética', 'Escreva, desenhe, faça diagramas e exiba informações com este quadro de escrita reutilizável. É ótimo para organizar a semana, fazer anotações diárias e/ou partilhar recados. Superfície de aço lacado magnético de limpeza a seco com 10 anos de garantia, para uso regular. Estrutura em liga Tech em preto ou cinza que combina bem com qualquer espaço devido ao seu design minimalista e leve.',
    99.90, 6398765432108),
    ('SKU00007', 'Marcador para quadros brancos', 'O marcador para quadros brancos Edding 360 possui tinta de pigmentos com odor reduzido e permite escrever de forma suave em todos os tipos de quadros brancos e de outras superfícies não porosas.',
    2.19, 9254321098765),
    ('SKU00008', 'FIRMO Caderno Clássico', 'O clássico caderno da Firmo está de regresso. Apontou momentos do dia a dia de gerações inteiras. Nas escolas marcou faltas de material e apontou sumários. Acertou as contas de muitas mercearias, picou o ponto de muitos trabalhadores e registou em ata muitas reuniões importantes. Agora surge renovado, com papel creme, elástico, bolsa para documentos na contracapa e fita de marcação.',
    12.99, 8424221398525);

INSERT INTO contains (order_no, SKU, qty)
VALUES
    (1,'SKU00003', 1),
    (2,'SKU00005', 10),
    (3,'SKU00006', 2),
    (3,'SKU00008', 4),
    (4,'SKU00001', 2),
    (4,'SKU00002', 1),
    (2,'SKU00006', 2),
    (2,'SKU00007', 1),
    (3,'SKU00001', 4),
    (3,'SKU00002', 5),
    (4,'SKU00003', 1),
    (5,'SKU00001', 2),
    (5,'SKU00002', 1),
    (6,'SKU00003', 1),
    (7,'SKU00005', 10),
    (7,'SKU00006', 2),
    (8,'SKU00007', 1),
    (9,'SKU00001', 4),
    (9,'SKU00002', 5),
    (10,'SKU00003', 1),
    (10,'SKU00004', 1),
    (10,'SKU00005', 1),
    (10,'SKU00006', 5),
    (10,'SKU00007', 29),
    (11,'SKU00004', 1),
    (12,'SKU00005', 1),
    (12,'SKU00006', 5),
    (11,'SKU00007', 29);

INSERT INTO supplier (TIN, name, address, SKU, date)
VALUES
    ('102344678', 'Creative Findings, LLC', 'Rua Dr. Michael Freitas, 17, 2400-001 Leiria', 'SKU00001', '2022-06-10'),
    ('104349678', 'LapiShop LDA', 'Rua Vasco da Gama, 9, 2400-001 Leiria', 'SKU00006', '2023-02-02'),
    ('105649434', 'LClassic Packaging Corp', 'Rua dos Pensadores, 2, 2615-011 Alverca', 'SKU00008', '2023-01-01');

INSERT INTO delivery (address, TIN)
VALUES
    ('Rua da Amizade, 5, 2660-503 Loures', '102344678'),
    ('Rua da Amizade, 5, 2660-503 Loures', '104349678'),
    ('Rua da Amizade, 5, 2660-503 Loures', '105649434');

## 1. Restrições de Integridade

Apresente o código para implementar as seguintes restrições de integridade, se necessário, com recurso a extensões procedimentais SQL (Stored Procedures e Triggers):

(RI-1) Nenhum empregado pode ter menos de 18 anos de idade

In [ ]:
%%sql

ALTER TABLE employee ADD CONSTRAINT age_check CHECK (bdate <= current_date - INTERVAL '18 years');

(RI-2) Um 'Workplace' é obrigatoriamente um 'Office' ou 'Warehouse' mas não pode ser ambos

In [ ]:
%%sql

DROP TRIGGER IF EXISTS check_workplace ON workplace;

-- TODO restrição da remoção

CREATE OR REPLACE FUNCTION check_address_in_other_table() RETURNS TRIGGER AS
$$
BEGIN
    IF EXISTS (
        SELECT *
        FROM warehouse
        WHERE address = NEW.address
    ) THEN
        IF EXISTS (
            SELECT *
            FROM office
            WHERE address = NEW.address
        ) THEN
            RAISE EXCEPTION 'Address exists in both warehouse and office.';
        END IF;
    ELSE
        IF NOT EXISTS (
            SELECT *
            FROM office
            WHERE address = NEW.address
        ) THEN
            RAISE EXCEPTION 'Address does not exist in warehouse or office.';
        END IF;
    END IF;

    RETURN NEW;
END
$$ LANGUAGE plpgsql;

CREATE CONSTRAINT TRIGGER check_workplace AFTER INSERT OR UPDATE ON workplace DEFERRABLE INITIALLY DEFERRED
    FOR EACH ROW EXECUTE FUNCTION check_address_in_other_table();

(RI-3) Uma 'Order' tem de figurar obrigatoriamente em 'Contains'.

In [ ]:
%%sql
-- (RI-3)


## 2. Consultas SQL

Apresente a consulta SQL mais sucinta para cada uma das seguintes questões

1) Qual o número e nome do(s) cliente(s) com maior valor total de encomendas pagas?  

In [ ]:
%%sql

WITH    total_by_customer AS (
            SELECT cust_no, c.name AS cust_name, SUM(qty*price) AS cust_total FROM
            customer c
            INNER JOIN pay
                       USING (cust_no)
            INNER JOIN contains
                       USING (order_no)
            INNER JOIN product
                       USING (SKU)
            GROUP BY cust_no, cust_name
        )

SELECT cust_no, cust_name FROM total_by_customer
    WHERE cust_total >= ALL(
        SELECT cust_total FROM total_by_customer
    );

2. Qual o nome dos empregados que processaram encomendas em todos os dias de 2022 em que houve encomendas?

In [ ]:
%%sql
SELECT name FROM (
    SELECT DISTINCT name, ssn FROM employee e1
    WHERE NOT EXISTS (
        SELECT date FROM orders
            WHERE EXTRACT(YEAR FROM date) = 2022
        EXCEPT
        SELECT date FROM
            employee e2
            INNER JOIN process
                       USING (ssn)
            INNER JOIN orders
                       USING (order_no)
            WHERE e2.ssn = e1.ssn
    )
) AS employees;

3. Quantas encomendas foram realizadas mas não pagas em cada mês de 2022?

In [ ]:
%%sql

WITH unpaid_orders AS (
    SELECT order_no, EXTRACT(MONTH FROM date) AS month FROM orders o
    WHERE EXTRACT(YEAR FROM date) = 2022
        AND NOT EXISTS (
            SELECT order_no FROM pay p
            WHERE p.order_no = o.order_no
        )
),

months AS (
    SELECT generate_series(1, 12) AS month
)

SELECT month, COUNT(order_no) FROM 
    months
    LEFT JOIN unpaid_orders
              USING (month) 
    GROUP BY month
    ORDER BY month;

## 3. Vistas

Crie uma vista que resuma as informações mais importantes sobre as vendas de produtos, combinando informações de diferentes tabelas do esquema de base de dados. A vista deve ter o seguinte esquema:

product_sales(sku, order_no, qty, total_price, year, month, day_of_month, day_of_week, city)

In [ ]:
%%sql

DROP VIEW IF EXISTS product_sales;

CREATE VIEW product_sales AS
SELECT sku, order_no, qty, qty*price as total_price,
    EXTRACT(YEAR FROM date) AS year,
    EXTRACT(MONTH FROM date) AS month,
    EXTRACT(DAY FROM date) AS day_of_month,
    EXTRACT(DOW FROM date) AS day_of_week,
    SUBSTRING(address, '[[:digit:]]{4}-[[:digit:]]{3} (.+)$') AS city
FROM product INNER JOIN contains USING (sku)
    INNER JOIN orders USING (order_no)
    INNER JOIN customer USING (cust_no);

## 4. Desenvolvimento de Aplicação

### Explicação da arquitetura da aplicação web, incluindo um link para uma versão de trabalho e as relações entre os vários ficheiros na pasta web/arquivos

...

## 5. Consultas OLAP

Usando a vista desenvolvida para a Questão 3, escreva duas consultas SQL que permitam analisar:

1. As quantidade e valores totais de venda de cada produto em 2022, globalmente, por cidade, por mês, dia do mês e dia da semana

In [ ]:
%%sql
-- SELECT ...

2. O valor médio diário das vendas de todos os produtos em 2022, globalmente, por mês e dia da semana

In [ ]:
%%sql
-- SELECT ...

## 6. Índices

Indique, com a devida justificação, que tipo de índice(s), sobre qual(is) atributo(s) e sobre qual(is) tabela(s) faria sentido criar, de forma a agilizar a execução de cada uma das seguintes consultas: 

### 6.1
SELECT order_no
FROM orders 
JOIN contains USING (order_no) 
JOIN product USING (SKU) 
WHERE price > 50 AND 
EXTRACT(YEAR FROM date) = 2023

### Tipo de Índice, Atributos & Justificação

...

### 6.2
SELECT order_no, SUM(qty*price)
FROM contains 
JOIN product USING (SKU) 
WHERE name LIKE ‘A%’ 
GROUP BY order_no;

### Tipo de Índice, Atributos & Justificação

...